<a href="https://colab.research.google.com/github/ngttnguyen/datacracy-atom-NguyenThaoNguyen/blob/main/NguyenThaoNguyen_ATOM_Home_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import email, smtplib, ssl # to automate email
import email as mail
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import requests, json, os # to pull API, and work with json
import datetime as dt # to work with date, time
from bs4 import BeautifulSoup # to work with web scrapping (HTML)
import pandas as pd # to work with tables (DataFrames) data
from IPython.core.display import HTML # to display HTML in the notebook

In [ ]:
# Email Bot function
# init email with header
def init_emaiWithlHeader(receiver, subject, sender = None):
    if sender is None:
        sender = os.environ['SENDER_EMAIL']
    email = MIMEMultipart()
    email['From'] = sender
    email['To'] = receiver
    email['Subject'] = subject 
    return email
#def compose_emailBody(email,text):
    
# Function of Email Bot
def send_email(receiver, subject, part1, part2, part3, sender = None, password = None, smtp_server = None,port = None):
    if sender is None:
        sender = os.environ['SENDER_EMAIL']
    if password is None:
        password = os.environ['PWD_EMAIL']
    if smtp_server is None:
        smtp_server = "smtp.gmail.com"      
    if port is None:
        port = 587
    # Compose email    
    email = init_emaiWithlHeader(receiver, subject,sender)
    email.attach(part1)
    email.attach(part2)
    email.attach(part3)
    try:
           # (3) Create SMTP session for sending the mail
        session = smtplib.SMTP(smtp_server, port) #use gmail with port
        session.starttls() #enable security
        session.login(sender, password) #login with mail_id and password
        text = email.as_string()
        session.sendmail(sender, receiver, text)
        print("DONE! Mail Sent from {} to {}".format(sender, receiver))
    except Exception as e:
        # Print any error messages to stdout
        print(e)
        print("Mait NOT SENT to {}".format(receiver))
    finally:
        session.quit()

In [ ]:
### WEATHER API
## Function get weather API from a weather site
def get_weatherAPI(city, lang = None, API_KEY = None):
    if lang is None:
        lang = "en"
    if API_KEY is None:
        API_KEY = os.environ['WEATHER_API_KEY']
        
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
       
    # upadting the URL
    # api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}
    URL = BASE_URL + "q=" + city + "&lang=" + lang + "&appid=" + API_KEY
    
    # HTTP request
    try:
        response =  requests.get(URL)       
        return(response)
    except Exception as e:
        print(e)

## Function get desired information from weather API
# output: information in dictionary format
def get_dicWeatherReport(response):
    if (response.status_code == 200):
        # getting data in the json format
        data = response.json()
       # getting the main dict block
        main = data['main']
       # weather report
        report = data['weather']
        city = data['name'] 
         
        detail_report ={}
        date_report = dt.datetime.now().strftime("%d/%m/%Y")
        detail_report['date'] = date_report
        detail_report['city'] = data['name']
        detail_report['description'] = report[0]['description']
        detail_report['temperature'] = main['temp']
        detail_report['humidity'] = main['humidity']
        detail_report['latitute'] = data['coord']['lat']
        detail_report['longtitute'] = data['coord']['lon']
        
        return(detail_report)
    else:
       # showing the error message
        print("Error in the HTTP request")
        print(response)

## Function get desired information from weather API        
# output: string information of weather
def get_WeatherInfo(response):       
    if (response.status_code == 200):
        # getting data in the json format
        data = response.json()
       # getting the main dict block
        main = data['main']
        city = data['name']
       # getting temperature
        temperature = main['temp']
   # getting the humidity
        humidity = main['humidity']
   # getting the pressure
        pressure = main['pressure']
   # weather report
        report = data['weather']
    # Longtitute and latitue
        lat = data['coord']['lat']
        lon = data['coord']['lon']
        description = report[0]['description']
        
        output ="""The Weather of {} at ({},{}) today:\nTemperature: {}\nHumidity: {}\nPressure: {}\nWeather Report: {}""".format(city,lon,lat,temperature,humidity,pressure,description)
        return(output)              
    else:
       # showing the error message
        print("Error in the HTTP request")
        print(response)
    

In [ ]:
## WEB SCRAPPING & HTML Function

# Funtion request Content
def get_WebContent(URL= None):
    if URL is None:
        URL="https://en.wikipedia.org/wiki/COVID-19"
    try:
        response =  requests.get(URL)       
        return(response)
    except Exception as e:
        print(e)

def get_News(response):       
    if(response.status_code == 200):
        soup = BeautifulSoup(response.content, 'html.parser')
        head = soup.find('head')
        title = head.title.string
        illness = soup.find(id="firstHeading").string

        p = soup.find_all(title = "Symptoms of COVID-19")
        symptons = p[0].string
        
        output = "{}\n{} has the following symptons: {}".format(title, illness, symptons)
        return(output)
    else:
        print(f"Could not get the content.\n HTTP {response.reason_phrase} {response.status_code}")

In [ ]:
def read_txtInfo(infoFile):
    try:
        with open(infoFile,'r') as f:
            info = f.read()
            f.close()
            return(info)
    except Exception as e:
        print(e)

def write_txtInfo(infoFile,text):
    with open(infoFile,'w') as f:
        f.write(text)
        f.close()
    print(f"DONE! Output in {infoFile} file")

# #Receiver list
# get data form csv file
# input: path of csv file
# output: dataFrame
def read_reveiverList(csvFile):
    try:
        receiverList = pd.read_csv(csvFile)
        return (receiverList)
    except Exception as e:
        print(e)
    

In [ ]:
def initText1(receiver):
    text1 = """
    <html>
        <p><strong><span style="font-size: 18px;">Dear {},</span></strong></p>
        <p>How are you today?</p>
        <p>Here is the you want.</p>
    </html>
    """.format(receiver)
    return(text1)

def initText2():
    text2 = """
    <html>
    Email sent at <b>{}</b><br>
    </html>
    """.format(dt.datetime.now().strftime("%d/%m/%Y, %H:%M:%S"))#format(dt.datetime.now().isoformat())
    return(text2)

    

In [ ]:
##### ----------------MAIN------
# load environment variables
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

os.environ['SENDER_EMAIL'] = json_data['SENDER_EMAIL'] ## INPUT: Your Email
os.environ['PWD_EMAIL'] = json_data['PWD_EMAIL'] ## INPUT: Your App Password
os.environ['WEATHER_API_KEY'] = json_data['WEATHER_API_KEY'] ## INPUT: OpenWeather API Key
print('DONE! Get Env. Variable')

# Init emailbot variables 
# 1. use gmail with port
smtp_server = "smtp.gmail.com"
port = 587
sender= os.environ['SENDER_EMAIL']
password = os.environ['PWD_EMAIL'] 

subject_weatherEmail = "Weather report from NguyenNguyen"
subject_newsEmail = "News from NguyenNguyen"
wikiUrl = "https://en.wikipedia.org/wiki/COVID-19"
content = 'Weather' #weather
news = "News"# wiki

#2. Create Body part
text2 = initText2()
part2 = MIMEText(text2, 'html')

# 3. Load email list from CSV file
receiver_filename = 'email_list.csv'
receiverList = read_reveiverList(receiver_filename)
receiver_emails = receiverList['email']
receiver_typeContents = receiverList['contents']
receiver_cities = receiverList['city']
print(receiver_emails )

#4. Send mail to user list
for i in range(0,len(receiver_emails)):
    if(receiver_typeContents[i] == news):     
        #4. Get Wiki Content       
        response = get_WebContent(wikiUrl)
        info = get_News(response)
        subject = subject_newsEmail
        
        info = wikiInfo
    else:
        #3. Get weather API
        city = receiver_cities[i]
        response = get_weatherAPI(city)
        info = get_WeatherInfo(response)
        subject = subject_weatherEmail
   
    
    text1 = initText1(receiver_emails[i])
    part1 = MIMEText(text1, 'html')
    part3 = MIMEText(info, 'plain')
    
    send_email(receiver_emails[i], subject, part1, part2, part3)

print("DONE!")
                 

DONE! Get Env. Variable
0           ngttnguyen@gmail.com
1        hoangnguyena2@yahoo.com
Name: email, dtype: object
DONE! Mail Sent from ntt.nguyen8381@gmail.com to     ngttnguyen@gmail.com
DONE! Mail Sent from ntt.nguyen8381@gmail.com to     hoangnguyena2@yahoo.com
DONE!
